In [1]:
#Utility
import urllib
import os
import datetime

#Data Science
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

#Google API
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta2

## Mobility

In [20]:
#Import mobility from url
url = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'
mobility = pd.read_csv(url)

#Transform mobility to time series
#startdate = datetime.datetime.strptime('2020-02-15','%Y-%m-%d').date()
#enddate = datetime.datetime.strptime('2020-04-11','%Y-%m-%d').date()

filterlist = ['retail_and_recreation_percent_change_from_baseline',
'grocery_and_pharmacy_percent_change_from_baseline',
'parks_percent_change_from_baseline',
'transit_stations_percent_change_from_baseline',
'workplaces_percent_change_from_baseline',
'residential_percent_change_from_baseline']

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
#Get ONLY US mobility
country = mobility['country_region'].value_counts()
countrylist = country.index.to_list()
usmobility = mobility[mobility['country_region']=='United States']

#Change column names
usmobility = usmobility.rename(columns={"sub_region_1":"Province_State", "sub_region_2": "Admin2"})

#Split "county" text out of county
countylist = []
txt = usmobility['Admin2'].to_list()
for i in txt:
    i = str(i)
    countylist.append(i.replace(' County', ''))
usmobility['Admin2'] = countylist

usmobility['Lookup'] = usmobility['Admin2'] + usmobility['Province_State'] + usmobility['date']
lookuplist = usmobility['Lookup'].to_list()
usmobility = usmobility.transpose()
usmobility.columns = lookuplist
usmobility = usmobility.transpose()

#usmobility.loc[:,filterlist].astype(float)

In [23]:
filterlist1 = filterlist.copy()
filterlist1.append('Lookup')
df = usmobility.loc[:,filterlist1]
usmobility = df.fillna(method='ffill')

## Cases from John Hopkins

In [27]:
#Import confirmed, deaths and recovered cases from url
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
#recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

In [28]:
#Formating confirmed as df ready for import for maindf
df = confirmed.copy()
xlist = ['UID', 'iso2', 'iso3', 'code3','Country_Region', 'Lat', 'Long_','Combined_Key']

df['Lookup'] = df['Admin2'] + df['Province_State']
includelist = [x for x in df.columns if x not in xlist]
datelist = includelist.copy()
datelist.pop(0)
datelist.pop(0)
datelist.remove('Lookup')
df = df.loc[:,includelist]

#Transpose and change index
lookuplist = df['Lookup'].to_list()
df = df.transpose()
df.columns = lookuplist
df = df.transpose()

In [29]:
#Formating deaths as df1 ready for import for maindf
df1 = deaths.copy()
xlist = ['UID', 'iso2', 'iso3', 'code3','Country_Region', 'Lat', 'Long_','Combined_Key']

df1['Lookup'] = df1['Admin2'] + df1['Province_State']
includelist = [x for x in df1.columns if x not in xlist]
datelist = includelist.copy()
datelist.pop(0)
datelist.pop(0)
datelist.pop(0)
datelist.pop(0)
datelist.remove('Lookup')
df1 = df1.loc[:,includelist]

#Transpose and change index
lookuplist = df1['Lookup'].to_list()
df1 = df1.transpose()
df1.columns = lookuplist
df1 = df1.transpose()

#Create a lookup list to loop over for maindf
lookuplist = df1['Lookup'].value_counts().index.to_list()
lookuplist = sorted(lookuplist)

## Contructing Main Dataframe

In [30]:
#Construction of maindf
colnames = ['Lookup','State','County','FIPS','Population','Date','Confirmed','Deaths']
#'retail_and_recreation_percent_change_from_baseline',
#'grocery_and_pharmacy_percent_change_from_baseline',
#'parks_percent_change_from_baseline',
#'transit_stations_percent_change_from_baseline',
#'workplaces_percent_change_from_baseline',
#'residential_percent_change_from_baseline']

maindf = pd.DataFrame(index=range(0,len(lookuplist)*len(datelist)), columns=colnames)

#Start loop for confirmed
j = 0
for i in lookuplist:
    testdf = df.loc[i,:]
    testdf1 = df1.loc[i,:]
    
    #Confirmed cases
    timeseries = testdf[3:-1]
    date = timeseries.index.to_list()
    timeseries = timeseries.to_list()
    length = len(timeseries)
    
    #Deaths
    timeseries1 = testdf1[4:-1].to_list()
    
    maindf.iloc[j:(j+length),colnames.index('Lookup')] = i
    maindf.iloc[j:(j+length),colnames.index('State')] = testdf[2]
    maindf.iloc[j:(j+length),colnames.index('County')] = testdf[1]
    maindf.iloc[j:(j+length),colnames.index('FIPS')] = testdf[0]
    maindf.iloc[j:(j+length),colnames.index('Population')] = testdf1[3]
    maindf.iloc[j:(j+length),colnames.index('Date')] = date
    maindf.iloc[j:(j+length),colnames.index('Confirmed')] = timeseries
    maindf.iloc[j:(j+length),colnames.index('Deaths')] = timeseries1
    j = j+length
    
maindf.loc[:,'Date1'] = pd.to_datetime(maindf['Date'],format='%m/%d/%y')
#test = maindf['Date1'].astype(str).tolist()
maindf['Lookup'] = maindf['Lookup'] + maindf['Date1'].astype(str).tolist()

labels = maindf['Lookup'].to_list()
maindf = maindf.transpose()
maindf.columns = labels
maindf = maindf.transpose()

In [31]:
#Merge maindf and mobility to finaldf
finaldf = maindf.merge(usmobility, left_on='Lookup', right_on='Lookup')

,Lookup,State,County,FIPS,Population,Date,Confirmed,Deaths,Date1,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
142903,WestonWyoming2020-02-16,Wyoming,Weston,56045,6927,2/16/20,0,0,2020-02-16,-28.0,-14.0,-63.0,2.0,-40.0,15.0
142904,WestonWyoming2020-02-17,Wyoming,Weston,56045,6927,2/17/20,0,0,2020-02-17,-28.0,-14.0,-63.0,2.0,-37.0,15.0
142905,WestonWyoming2020-02-18,Wyoming,Weston,56045,6927,2/18/20,0,0,2020-02-18,-28.0,-7.0,-63.0,2.0,2.0,15.0
142906,WestonWyoming2020-02-19,Wyoming,Weston,56045,6927,2/19/20,0,0,2020-02-19,-28.0,-2.0,-63.0,2.0,0.0,15.0
142907,WestonWyoming2020-02-20,Wyoming,Weston,56045,6927,2/20/20,0,0,2020-02-20,-28.0,0.0,-63.0,2.0,10.0,15.0
142908,WestonWyoming2020-02-21,Wyoming,Weston,56045,6927,2/21/20,0,0,2020-02-21,-28.0,-2.0,-63.0,2.0,-14.0,15.0
142909,WestonWyoming2020-02-24,Wyoming,Weston,56045,6927,2/24/20,0,0,2020-02-24,-28.0,-2.0,-63.0,2.0,-4.0,15.0
142910,WestonWyoming2020-02-25,Wyoming,Weston,56045,6927,2/25/20,0,0,2020-02-25,-28.0,-9.0,-63.0,2.0,2.0,15.0
142911,WestonWyoming2020-02-26,Wyoming,Weston,56045,6927,2/26/20,0,0,2020-02-26,-28.0,-4.0,-63.0,2.0,-6.0,15.0
142912,WestonWyoming2020-02-27,Wyoming,Weston,56045,6927,2/27/20,0,0,2020-02-27,-28.0,-4.0,-63.0,2.0,0.0,15.0
